In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

In [2]:
# imports
%load_ext autoreload
%autoreload 2

In [3]:
import importlib
from tqdm.notebook import tqdm
import pickle

# import sys
# import os
# import copy
# import pathlib
# import time
# import gc
# from tqdm.notebook import tqdm, trange
# import numpy as np
# import matplotlib.pyplot as plt
# import seaborn as sns
# import scipy

# import torch
# import torchvision
# import torchvision.transforms as transforms

# import sklearn
# from sklearn.model_selection import train_test_split

# %matplotlib inline
# %matplotlib inline

In [4]:
base_dir = '/Users/josh/Documents/'

In [5]:
# Data Loading Function
import sys
import torch
import numpy as np
sys.path.append(f'{base_dir}/github_repos')
dir_folders = f'{base_dir}/label_data'
from basic_neural_processing_modules import h5_handling, pickle_helpers

def load_h5(path, h5_keys):
    base_data = h5_handling.simple_load(path=path)
    h5_subsets = [base_data[key] for key in h5_keys]
    return torch.as_tensor(np.concatenate(h5_subsets, axis=0), dtype=torch.float32, device='cpu')

def drop_nan_imgs(rois):
    ROIs_without_NaNs = torch.where(~torch.any(torch.any(torch.isnan(rois), dim=1), dim=1))[0]
    return rois[ROIs_without_NaNs]

In [6]:
# Load Data

path_to_h5 = f'{base_dir}/label_data/masks_20211202_balanced.h5'
h5_keys = ['RHmasks']

data = load_h5(path_to_h5, h5_keys)
data = drop_nan_imgs(data)

In [7]:
print(data.shape[0], 'examples loaded.')

217802 examples loaded.


In [8]:
# data = data[:2048]
data = data[:data.shape[0]//100]

In [9]:
# Instantiate Model
model_name = 'ResNet18_simCLR_model_202112078_EOD_transfmod=norm'
model_file = importlib.import_module(model_name)

### Have a .py file for each class definitions
### Includes all architectures
    # Within model.py — model = ModelTackOn(model_chopped, base_model_frozen, pre_head_fc_sizes=[1024, 512], post_head_fc_sizes=[64], classifier_fc_sizes=[len(np.unique(y_labeled_train))])
    # Every .pth file will be associated with a single .py file that will instantiate the model / ModelTackOn

model = model_file.get_model(model_name)
# ?? model.eval();
model.eval();

In [10]:
# Create Data Sets / Data Loaders

# Include necessary transformations to push any data through -- no augmentations

dataset, dataloader = model_file.get_dataset_dataloader(data)

In [11]:
def dataloader_to_latents(dataloader, model, DEVICE='cpu'):
    def subset_to_latents(data):
        return model.get_head(model.base_model(data[0][0].to(DEVICE))).detach().cpu()
    return torch.cat([subset_to_latents(data) for data in tqdm(dataloader)], dim=0)

In [12]:
# Get Model Latents
latents = dataloader_to_latents(dataloader, model).numpy()

In [13]:
def load_classifier_model(classifier_name):
    with open(classifier_name, 'rb') as classifier_model_file:
        classifier = pickle.load(classifier_model_file)
    return classifier

In [14]:
# Load Logistic Model

### Separate directory thing
classifier_model = load_classifier_model('./logreg_model_0.01.pkl')

In [15]:
from GCaMP_ROI_classifier.new_stuff import util
def get_returns(latents, classifier_model):
    ret = {}
    ret['latents'] = latents
    ret['proba'] = classifier_model.predict_proba(latents)
    ret['preds'] = np.argmax(ret['proba'], axis=-1)
    ret['uncertainty'] = util.loss_uncertainty(torch.as_tensor(ret['proba']), temperature=1, class_value=None).detach().cpu().numpy()
    
#     h5_prm = {}
    params = classifier_model.get_params()
#     for k in params:
#         h5_prm[k] = np.array(params[k]).astype(np.float64)
#     ret['params'] = h5_prm
    ret['params'] = params
    return ret

In [16]:
# Predict ROIs — Save to File

### Save the activations 
### 

### Outputs -- Latents for every ROI
### Activations of the classifier for each example
### Probabilities of activations for each example
### Predictions for each example
### L2 confidence value for each example

### Details about the run
### Throw all of the output arrays into an H5
### If you can save strings -- save the names of the nets &
### parameters of logistic regression (C/maxiter/etc.)
### everything that generated the results. Save logistic
### regression object as pickled file. -- get parameters
### function of the logistic regression class
### If it's possible to peel the parameters of the object --
### get_params() -- take those and throw them into a nested part of H5.

### Anything else I can think of.

preds = classifier_model.predict_proba(latents)
preds

array([[0.0013181 , 0.00416862, 0.7061961 , 0.28831718],
       [0.00216429, 0.00909407, 0.79564499, 0.19309665],
       [0.10365772, 0.71608473, 0.17253274, 0.00772481],
       ...,
       [0.00618561, 0.09349354, 0.78144143, 0.11887942],
       [0.01575943, 0.01893448, 0.49226415, 0.47304194],
       [0.04449105, 0.19101393, 0.48540233, 0.27909269]])

In [17]:
ret = get_returns(latents, classifier_model)
ret

{'latents': array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.        , 0.        , 0.        , ..., 0.        , 0.19630903,
         0.        ],
        [1.3864001 , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        ...,
        [0.32947278, 0.        , 0.        , ..., 6.623164  , 0.        ,
         0.        ],
        [0.        , 4.399821  , 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.        , 4.686661  , 0.        , ..., 0.        , 0.        ,
         0.        ]], dtype=float32),
 'proba': array([[0.0013181 , 0.00416862, 0.7061961 , 0.28831718],
        [0.00216429, 0.00909407, 0.79564499, 0.19309665],
        [0.10365772, 0.71608473, 0.17253274, 0.00772481],
        ...,
        [0.00618561, 0.09349354, 0.78144143, 0.11887942],
        [0.01575943, 0.01893448, 0.49226415, 0.47304194],
        [0.04449105, 0.19101393, 0.48540233, 0.27909269]]),
 'preds': a

In [18]:
# write_dict_to_h5( , ret , write_mode='w-', show_item_tree_pref=True)
# h5_handling.simple_save(ret, path='./tmp_dict_h5.h5', write_mode='w', verbose=False)

pickle_helpers.simple_save(ret, './tmp_dict_pkl.pkl')